# Imports

In [1]:
import pandas as pd
import numpy as np
import gc
from warnings import filterwarnings
from sklearn.metrics import mean_squared_error

train_path = r'D:/KaggleTabular/August/train.csv'
test_path = r'D:/KaggleTabular/August/test.csv'
submission_path = r'D:/KaggleTabular/August/sample_submission.csv'

rs = 69420
filterwarnings('ignore')

In [2]:
train = pd.read_csv(train_path, index_col=0)
train

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
id,,,,,,,,,,,,,,,,,,,,,
0,-0.002350,59,0.766739,-1.350460,42.27270,16.68570,30.35990,1.267300,0.392007,1.09101,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.296580,10.489800,15
1,0.784462,145,-0.463845,-0.530421,27324.90000,3.47545,160.49800,0.828007,3.735860,1.28138,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.364700,1.143830,3
2,0.317816,19,-0.432571,-0.382644,1383.26000,19.71290,31.10260,-0.515354,34.430800,1.24210,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.696790,12.305500,6
3,0.210753,17,-0.616454,0.946362,-119.25300,4.08235,185.25700,1.383310,-47.521400,1.09130,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.456450,10.028800,2
4,0.439671,20,0.968126,-0.092546,74.30200,12.30650,72.18600,-0.233964,24.399100,1.10151,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.784850,7.071970,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,0.923980,6,0.663212,-0.055120,6.61768,1.26619,40.44790,0.852375,25.522400,1.12294,...,3517.97000,18.534200,1.41936,0.607504,4.950780,2.853420,0.961882,1.068010,1.437650,11
249996,0.243556,7,-0.557062,1.333470,-54.88610,17.58310,212.96400,-1.204750,-128.385000,1.19334,...,1945.81000,19.482200,1.60072,-0.154877,5.693280,1.994540,0.760925,1.427880,1.226720,5
249997,0.046023,53,0.462863,0.704034,2062.94000,14.58160,11.42710,-0.509812,80.818000,1.20383,...,-28.46890,11.008200,1.61080,-0.350927,27.986900,1.012690,0.830533,1.005470,0.455332,1


In [3]:
# No NULL values
# nullvaluecheck = pd.DataFrame(train.isna().sum().sort_values(ascending=False)*100/train.shape[0],columns=['missing %']).head(10)
# nullvaluecheck.style.background_gradient(cmap='PuBu')

In [4]:
X = train.iloc[:, :-1].values
y = train.iloc[:, -1].values

X.shape, y.shape

((250000, 100), (250000,))

# Default LightGBM Model

In [5]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=rs)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((225000, 100), (25000, 100), (225000,), (25000,))

In [7]:
from sklearn.preprocessing import RobustScaler
sc = RobustScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
# GPU training is slightly faster but accuracy slightly lower
lgbm = LGBMRegressor(
    boosting_type='gbdt',
    metric='rmse',
    device='cpu',
    learning_rate=0.01,
    num_iterations=10000,
    random_state=rs
)

In [9]:
_ = gc.collect()

In [10]:
%%time
lgbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=50, verbose=1000)

Training until validation scores don't improve for 50 rounds
[1000]	valid_0's rmse: 7.87303
[2000]	valid_0's rmse: 7.86561
Early stopping, best iteration is:
[1959]	valid_0's rmse: 7.86558
Wall time: 27.8 s


LGBMRegressor(device='cpu', learning_rate=0.01, metric='rmse',
              num_iterations=10000, random_state=69420)

In [11]:
y_pred = lgbm.predict(X_test)
mse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {round(mse, 5)}")

RMSE: 7.86558


In [12]:
# 7.86558

## Submission with Default LGBM

 Retrain the model with the full dataset to squeeze out all the performance

In [13]:
from sklearn.preprocessing import RobustScaler
sc = RobustScaler()

X = sc.fit_transform(X)

In [14]:
# GPU training is slightly faster but accuracy slightly lower
lgbm = LGBMRegressor(
    boosting_type='gbdt',
    metric='rmse',
    device='cpu',
    learning_rate=0.01,
    num_iterations=2000,
    random_state=rs
)

In [15]:
_ = gc.collect()

In [16]:
%%time
lgbm.fit(X, y)

Wall time: 31.5 s


LGBMRegressor(device='cpu', learning_rate=0.01, metric='rmse',
              num_iterations=2000, random_state=69420)

In [17]:
submission = pd.read_csv(submission_path, index_col=0)
submission.head()

,loss
id,
250000,0
250001,0
250002,0
250003,0
250004,0


In [18]:
test = pd.read_csv(test_path, index_col=0)
test.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
id,,,,,,,,,,,,,,,,,,,,,
250000,0.812665,15,-1.239120,-0.893251,295.5770,15.87120,23.04360,0.942256,29.898000,1.11394,...,0.446389,-422.332,-1.44630,1.69075,1.059300,-3.010570,1.94664,0.529470,1.386950,8.78767
250001,0.190344,131,-0.501361,0.801921,64.8866,3.09703,344.80500,0.807194,38.421900,1.09695,...,0.377179,10352.200,21.06270,1.84351,0.251895,4.440570,1.90309,0.248534,0.863881,11.79390
250002,0.919671,19,-0.057382,0.901419,11961.2000,16.39650,273.24000,-0.003300,37.940000,1.15222,...,0.990140,3224.020,-2.25287,1.55100,-0.559157,17.838600,1.83385,0.931796,2.336870,9.05400
250003,0.860985,19,-0.549509,0.471799,7501.6000,2.80698,71.08170,0.792136,0.395235,1.20157,...,1.396880,9689.760,14.77150,1.41390,0.329272,0.802437,2.23251,0.893348,1.359470,4.84833
250004,0.313229,89,0.588509,0.167705,2931.2600,4.34986,1.57187,1.118300,7.754630,1.16807,...,0.862502,2693.350,44.18050,1.58020,-0.191021,26.253000,2.68238,0.361923,1.532800,3.70660


In [19]:
test = sc.transform(test)

In [20]:
submission['loss'] = lgbm.predict(test)
submission.head()

,loss
id,
250000,8.021386
250001,5.223572
250002,7.486557
250003,7.348433
250004,7.947850


In [21]:
submission.to_csv('submission.csv')

In [22]:
# This code will submit the model to the competition if you have installed kaggle
# !kaggle competitions submit -c tabular-playground-series-aug-2021 -f submission.csv -m "Full Trained Default LGBM"